# 13-2 Machine Learning Open Framework

In [1]:
# Package imports
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
# Our own private imports
import imp
import joy
imp.reload(joy)

%matplotlib inline
np.random.seed(1)   # a good practice for reproducibility and debugging

# The following code is used for hiding the warnings and 
# make this notebook clearer.
#import warnings
#warnings.filterwarnings('ignore')

/home/soohyun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Build CNN: TensorFlow

### 1. read MNIST data from Tensorflow

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### 2. Define variables and functions for operation in Tensorflow

In [3]:
import tensorflow as tf

x = tf.placeholder(tf.float32, [None, 784])

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

### 3. Build NN in Tensorflow
for learning MNIST data.  
input layer + two hidden layers + output layer.

In [4]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
dropout_ratio = tf.placeholder(tf.float32)

x_train = tf.reshape(x, [-1, 28, 28, 1])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_train, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, dropout_ratio)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_hat = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

### 4-1. Fit NN in Tensorflow
To learn the neural network, define variables that contain costs or losses to check the performance of the model.  
It also uses Cross Entropy to define the model's loss.

In [5]:
CE = tf.reduce_mean(
        -tf.reduce_sum(y * tf.log(y_hat), reduction_indices=[1])
     )
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(CE)
correct_prediction = tf.equal(tf.argmax(y_hat,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(10):
        batch = mnist.train.next_batch(32)
        if i%100 == 0:
            train_accuracy = accuracy.eval(session=sess, feed_dict={
                x:batch[0], y: batch[1], dropout_ratio: 1.0
            })
            print("step %d, training accuracy %g"%(i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y: batch[1], dropout_ratio: 0.2})
    
    acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, dropout_ratio: 1.0})
    print("Test accuracy: {}".format(acc))

step 0, training accuracy 0
Test accuracy: 0.29580000042915344


### 4-2. Fit NN in Tensorflow

In [6]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

y_hat = tf.nn.softmax(tf.matmul(x, W) + b)
y = tf.placeholder(tf.float32, [None, 10])
CE = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_hat), reduction_indices=[1]))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(CE)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(1000):
    batch_x, batch_y = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_x, y: batch_y})

for i in range(1000):
    batch_x, batch_y = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_x, y: batch_y})

### 5. Evaluate models in Tensorflow
The predicted value determined by tf.argmax(y_hat, 1) and tf.argmax(y, 1) compares the actual label with the value to determine if it was learned correctly.

In [7]:
correct_prediction = tf.equal(tf.argmax(y_hat,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print("Test accuracy: {}".format(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})))

Test accuracy: 0.9222000241279602


## Build CNN : Keras

### 1. read MNIST data from Keras

In [8]:
from keras.datasets import mnist
from keras import backend as K
from keras.utils import np_utils
import numpy as np

img_rows, img_cols = 28, 28

(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

Using TensorFlow backend.


### 2. Build NN in Keras
input layer + two hidden layers + output layer.

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# define the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# summarize the model
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 1,199,882
Trainable params: 1,199,882
Non-trainable params: 0
_________________________________________________________________


### 3. Compile a Neural Network Model in Keras
The neural network has Cross Entropy, and the optimizer uses Adadelta. It also compiles to measure accuracy.

In [10]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

### 4. Fit NN in Keras
Let the neural network model learn. By using checkpointer, save the model every epoch.

In [11]:
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='mnist.model.best.hdf5', 
                               verbose=1, save_best_only=True)
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test),
         callbacks=[checkpointer])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 83s 1ms/step - loss: 0.1936 - acc: 0.9408 - val_loss: 0.0723 - val_acc: 0.9770

Epoch 00001: val_loss improved from inf to 0.07230, saving model to mnist.model.best.hdf5
Epoch 2/10
60000/60000 [==============================] - 83s 1ms/step - loss: 0.0464 - acc: 0.9857 - val_loss: 0.0325 - val_acc: 0.9883

Epoch 00002: val_loss improved from 0.07230 to 0.03249, saving model to mnist.model.best.hdf5
Epoch 3/10
60000/60000 [==============================] - 83s 1ms/step - loss: 0.0293 - acc: 0.9907 - val_loss: 0.0295 - val_acc: 0.9901

Epoch 00003: val_loss improved from 0.03249 to 0.02949, saving model to mnist.model.best.hdf5
Epoch 4/10
60000/60000 [==============================] - 83s 1ms/step - loss: 0.0197 - acc: 0.9939 - val_loss: 0.0260 - val_acc: 0.9919

Epoch 00004: val_loss improved from 0.02949 to 0.02596, saving model to mnist.model.best.hdf5
Epoch 5/10
60000/60000 [==

### 5. Measure NN classification accuracy in Keras

In [12]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.03542636290869236
Test accuracy: 0.9919


## Build CNN : PyTorch

### 1. read MNIST data from Pytorch

In [13]:
import torch
from torchvision import datasets, transforms

use_cuda = not False and torch.cuda.is_available()
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
        batch_size=64, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, 
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
        batch_size=64, shuffle=True, **kwargs)

### 2. Build NN in PyTorch
input layer + two hidden layers + output layer.  
also build a forward pass with class.

In [14]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=2)
        self.conv1_drop = nn.Dropout2d(0.2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=2)
        self.conv2_drop = nn.Dropout2d(0.2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=2)
        self.conv3_drop = nn.Dropout2d(0.2)
        
        self.fc1 = nn.Linear(256, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(x)), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = F.relu(F.max_pool2d(self.conv3_drop(self.conv3(x)), 2))
        x = x.view(-1, 256)
        x = self.fc1(x)
        
        return F.log_softmax(x)

### 3. Define NN variables
epochs, learning_rate etc.

In [15]:
n_epochs = 10
learning_rate = 0.01
random_seed = 1
log_interval = 10

network = Net()
optimizer = optim.RMSprop(network.parameters(), lr=learning_rate)

### 4. Define a fitting method
Define a function to learn data from the neural network. Use zero_gradient with the optimizer and cross entropy to minimize the loss function.

In [16]:
def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            

### 5. Define a test method

In [17]:
def test():
    network.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        output = network(data)
        test_loss += F.cross_entropy(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
        test_loss /= len(test_loader.dataset)
    print('Test Accuracy {}%\n'.format(100.*correct / len(test_loader.dataset)))

### 6. Measure accuracy in PyTorch

In [22]:
for epoch in range(1, n_epochs+1):
    train(epoch)
    test()

/home/soohyun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 0.320818
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.111332
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.333163
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.525088
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.163675
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.193692
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.391280
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.224865
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.265631
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.283650


KeyboardInterrupt: 